In [1]:
import os
import sys
from typing import Optional, Any

from dotenv import load_dotenv, find_dotenv
import numpy as np

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

/media/D/miniconda/envs/llm_intro/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# make sure GOOGLE_API_KEY (or any other llm provider, must exactly the same)
_ = load_dotenv(find_dotenv())
# or put it into variable and pass it to llm model 
# google_api_key = os.getenv('GOOGLE_API_KEY') # pass this to the model creation

# Agent Concept

In [55]:
"""
agent that we use as example:
    1. get weather
    2. check time (could be in different timezone)
    3. check latest 10 posts of reddit/hacker news
"""
from datetime import datetime
import zoneinfo as zi
import requests
from langchain_core.tools import BaseTool
from langchain_core.tools import Tool, tool, StructuredTool
from pydantic import BaseModel, Field
from typing import Type, Optional


# define input with pydantic
class TimeZone(BaseModel):
    timezone: str =  Field(description="The timezone")

class Location(BaseModel):
    location: str = Field(description='Location to find weather')
    

@tool
def get_time(timezone: Optional[str] = 'Asia/Jakarta') -> str:
    """Get time in a specific timezone. Use Asia/Jakarta if not provided"""
    utc_now = datetime.utcnow()
    return "Current date: " + utc_now.astimezone(zi.ZoneInfo(timezone)).strftime('%Y-%m-%d %H:%M:%S')
@tool
def get_weather(location: Optional[str]='Jakarta') -> str:
    """Get the weather via API in a location (provided, Jakarta as default).
    A tool designed to gather accurate information on weather around the globe. 
    Useful for when you need to answer questions about the weather. 
    Input should be the location.
    """
    WEATHERAPI_API_KEY='8a6c94f591c4401c9e8152836251301'
    res = requests.get(f'https://api.weatherapi.com/v1/current.json?key=8a6c94f591c4401c9e8152836251301&q={location}&aqi=no')
    return "For {} temperature: {:.2F} and weather: {}".format(
        location,
        res.json()['current']['temp_c'],
        res.json()['current']['condition']['text']
        )
@tool
def get_weather_average(temperature: list[float]) -> float:
    """ Calculate the weather (temperature) average from `get_weather` function. Input is list of temperature. It could be called to get average weather of two cities or more."""
    return sum(temperature)/ len(temperature) if temperature else 0

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm_w_tools = llm.bind_tools([weather_tool, weather_difference_tool])

In [6]:
weather_tool.run('Jakarta')

'For Jakarta temperature: 28.20 and weather: Partly cloudy'

In [57]:
from langgraph.prebuilt import create_react_agent

# query = "How's the weather in Jakarta and Bandung? and what's the average?"
query = "What's the temperature Seoul, Tokyo, Jakarta and Kuala Lumpur right now? and what's the average of all cities?"
agent_executor = create_react_agent(model=llm, tools=[get_weather, get_weather_average])
message = agent_executor.invoke({
    'messages': [
        ('human', query)
    ]
})
message['messages'][-1].content

"The temperature in Seoul is -5.7 degrees Celsius and the weather is clear.  In Tokyo, it's 4.6 degrees Celsius and clear. Jakarta is 28 degrees Celsius with partly cloudy skies, and Kuala Lumpur is 25.2 degrees Celsius with light rain.\n\nThe average temperature across all four cities is 13.02 degrees Celsius.\n"

## Saving Checkpoint

In [63]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

agent_executor = create_react_agent(model=llm, tools=[get_weather, get_weather_average], checkpointer=memory)

# create configuration for Thread Id
config = {"configurable": {"thread_id": "test-thread"}}
message = agent_executor.invoke({
    'messages': [
        ('human', query)
    ]},
    config
)

In [66]:
message['messages'][-1]

AIMessage(content='The temperature in Seoul is -5.7°C and the weather is clear.  In Tokyo it is 5.1°C and clear. Jakarta is 28°C with partly cloudy skies, and Kuala Lumpur is 25.2°C with light rain.\n\nThe average temperature across all four cities is 13.15°C.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4c43c404-3eb3-4067-94ab-992a1479b9f6-0', usage_metadata={'input_tokens': 438, 'output_tokens': 77, 'total_tokens': 515, 'input_token_details': {'cache_read': 0}})

In [67]:
m = agent_executor.invoke({
    'messages': [
        ('human', 'What is the city that I ask previously?')
    ]},
    config
)

In [69]:
m['messages'][-1]

AIMessage(content='The cities you asked about previously were Seoul, Tokyo, Jakarta, and Kuala Lumpur.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-182cdece-2671-4c3e-a902-50d8436db378-0', usage_metadata={'input_tokens': 526, 'output_tokens': 18, 'total_tokens': 544, 'input_token_details': {'cache_read': 0}})